Я сделал target.csv (Count, IsTrain, IsTest, IsValidation) индекс совпадает с date.csv, пофиксил твою функцию с вычеслением глобальной нумерации недель, два твоих бага пофиксил - простые(видимо ты их случайно заккомитил), со сборкой средних у меня возникли запары, сам делай. Учитывай, что по трейну нужно считать признаки не только для самого трейна, но и для тестовой и валидационной выборки (вот с этим у меня проблемы возникли)

In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import sklearn.cross_validation

In [4]:
train = pd.read_csv('data_original/train.csv')
test = pd.read_csv('data_original/test.csv')
test['Count'] = None
frame = [train, test]
common = pd.concat(frame)
common.Date = common.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
common = common.sort_values(by=['Date'])

In [3]:
common.to_csv('data_original/common.csv', index=None)
train.to_csv('data_original/train.csv', index=None)
test.to_csv('data_original/test.csv', index=None)

# date.csv

In [4]:
date = pd.read_csv('data_original/common.csv')[['Date']]
date.Date = date.Date.apply(lambda x: pd.to_datetime(x))

In [5]:
start_date = date.iloc[0]['Date']
date['Year'] = date.Date.apply(lambda x: x.year)
date['Month'] = date.Date.apply(lambda x: x.month)
date['MonthDay'] = date.Date.apply(lambda x: x.day)
date['WeekDay'] = date.Date.apply(lambda x: x.weekday())

In [6]:
date['YearDay'] = date.Date.apply(lambda x: (x-pd.to_datetime(str(x.year)+'-01-01')).days)

In [7]:
date['ThroughMonth'] = date.Date.apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)

In [8]:
date['ThroughWeek'] = date.Date.apply(lambda x: int( ( (x - start_date).days + start_date.weekday()) / 7))
#теперь нет смещения, это могло вводить неоднозначноть, т.е. одной недели могли противоречить две фичи

In [9]:
date['ThroughDay'] = date.Date.apply(lambda x: (x - start_date).days)

In [10]:
date.to_csv('data_transform/date.csv', index=None)

In [11]:
date.head(25)

,Date,Year,Month,MonthDay,WeekDay,YearDay,ThroughMonth,ThroughWeek,ThroughDay
0,2011-10-06,2011,10,6,3,278,0,0,0
1,2011-10-07,2011,10,7,4,279,0,0,1
2,2011-10-09,2011,10,9,6,281,0,0,3
3,2011-11-15,2011,11,15,1,318,1,6,40
4,2011-12-08,2011,12,8,3,341,2,9,63
5,2011-12-09,2011,12,9,4,342,2,9,64
6,2011-12-10,2011,12,10,5,343,2,9,65
7,2011-12-11,2011,12,11,6,344,2,9,66
8,2011-12-13,2011,12,13,1,346,2,10,68
9,2011-12-14,2011,12,14,2,347,2,10,69


# target.csv

In [12]:
sizeValidationSample = 0.15

In [13]:
target = pd.read_csv('data_original/common.csv')[['Count']]
target['IsTest'] = target.Count
target['IsTrain'] = pd.Series(range(target.shape[0]))
target['IsValidation'] = pd.Series(range(target.shape[0]))

In [14]:
target['IsTest'] = target.IsTest.apply(lambda x: 1 if math.isnan(x) else 0)

In [15]:
withCountValue = target[target.IsTest == 0].index
indexTrain, indexValidation = sklearn.cross_validation.train_test_split(withCountValue, test_size=sizeValidationSample)

In [16]:
target['IsTrain'] = target.IsTrain.apply(lambda x: 1 if ((x in indexTrain) & (target.IsTest[x] == 0)) else 0)
target['IsValidation'] = target.IsValidation.apply(lambda x: 1 if ((x in indexValidation) & (target.IsTest[x] == 0)) else 0)

In [17]:
target.to_csv('data_transform/target.csv', index = None)

In [18]:
target.head(10)

,Count,IsTest,IsTrain,IsValidation
0,8,0,1,0
1,88,0,1,0
2,1,0,1,0
3,1,0,1,0
4,2,0,0,1
5,43,0,1,0
6,10,0,1,0
7,1,0,1,0
8,60,0,1,0
9,NaN,1,0,0


# city.csv


In [19]:
cities = pd.read_csv('data_original/common.csv')[['Point']].Point.unique()
city = pd.DataFrame()
city['Point'] = pd.Series(cities)
city['CityName'] = city.Point.apply(lambda x: '-'.join(x.split('-')[0:-1]))
city_names = city.CityName.unique()
city['CityID'] = city.CityName.apply(lambda x: np.where(city_names == x)[0][0]) 
city['BranchNumber'] = city.Point.apply(lambda x: x.split('-')[-1:][0])
# Москва 1 -> Москва
city['CityName'] = city.CityName.apply(lambda x: ('Москва' if x == 'Москва 1' else x))
city.to_csv('data_transform/city.csv', index = None)

In [20]:
# geo
city = pd.read_csv('data_transform/city.csv')
cities = city.CityName.unique()
geo = pd.read_csv('data_transform/geo.csv', names=['CityName', 'lat', 'lng'])
geo.CityName = pd.Series(cities)
city = pd.merge(city, geo, how='left', on='CityName')
city.to_csv('data_transform/city.csv', index=None)
city.head(10)

,Point,CityName,CityID,BranchNumber,lat,lng
0,Сыктывкар-1,Сыктывкар,0,1,61.647851,50.833903
1,Сыктывкар-2,Сыктывкар,0,2,61.647851,50.833903
2,Ухта-1,Ухта,1,1,63.567321,53.747159
3,Смоленск-1,Смоленск,2,1,54.790311,32.050366
4,Сарапул-1,Сарапул,3,1,56.453929,53.774218
5,Самара-1,Самара,4,1,53.241504,50.221246
6,Челябинск-1,Челябинск,5,1,55.164442,61.436843
7,Белгород-1,Белгород,6,1,50.599713,36.598262
8,Смоленск-2,Смоленск,2,2,54.790311,32.050366
9,Самара-2,Самара,4,2,53.241504,50.221246


In [17]:
# get date of first order for every branch
city = pd.read_csv('data_transform/city.csv')
common = pd.read_csv('data_original/common.csv')
common.Date = common.Date.apply(lambda x: pd.to_datetime(x))
groupped = common.groupby(['Point'])['Date'].min()
start_date = pd.DataFrame()
start_date['Point'] = groupped.index
start_date['FirstOrder'] = groupped.as_matrix()
city = pd.merge(city, start_date[['Point', 'FirstOrder']], how='left', on='Point')
city = city[city.columns.difference(['Unnamed: 0'])]
city.to_csv('data_transform/city.csv', index=None)
city.head()


,BranchNumber,CityID,CityName,FirstOrder,Point,lat,lng
0,1,0,Сыктывкар,2011-10-06,Сыктывкар-1,61.647851,50.833903
1,2,0,Сыктывкар,2012-05-03,Сыктывкар-2,61.647851,50.833903
2,1,1,Ухта,2012-07-17,Ухта-1,63.567321,53.747159
3,1,2,Смоленск,2012-08-28,Смоленск-1,54.790311,32.050366
4,1,3,Сарапул,2012-11-03,Сарапул-1,56.453929,53.774218


# internal_features.csv

In [19]:
src = pd.read_csv('data_transform/date.csv')
src[['Count', 'Point']] = pd.read_csv('data_original/common.csv')[['Count', 'Point']]
dst = pd.DataFrame() 

In [39]:
#tmp = pd.read_csv('data_transform/target.csv')
#tmp = pd.read_csv('data_transform/target.csv')[(tmp.IsTrain == True) | (tmp.IsValidation == True)]

#trainIndex = tmp.index

In [40]:
#src = pd.read_csv('data_transform/date.csv').iloc[trainIndex]
#src[['Count', 'Point']] = pd.read_csv('data_original/common.csv')[['Count', 'Point']].iloc[trainIndex]
#dst = pd.DataFrame() 

In [20]:
tmp = pd.read_csv('data_transform/target.csv')
validationIndex = tmp[tmp.IsValidation == True].index
src.loc[validationIndex, 'Count'] = None

In [21]:
src.head(10)

,Date,Year,Month,MonthDay,WeekDay,YearDay,ThroughMonth,ThroughWeek,ThroughDay,Count,Point
0,2011-10-06,2011,10,6,3,278,0,0,0,8.0,Сыктывкар-1
1,2011-10-07,2011,10,7,4,279,0,0,1,88.0,Сыктывкар-1
2,2011-10-09,2011,10,9,6,281,0,0,3,1.0,Сыктывкар-1
3,2011-11-15,2011,11,15,1,318,1,6,40,1.0,Сыктывкар-1
4,2011-12-08,2011,12,8,3,341,2,9,63,NaN,Сыктывкар-1
5,2011-12-09,2011,12,9,4,342,2,9,64,43.0,Сыктывкар-1
6,2011-12-10,2011,12,10,5,343,2,9,65,10.0,Сыктывкар-1
7,2011-12-11,2011,12,11,6,344,2,9,66,1.0,Сыктывкар-1
8,2011-12-13,2011,12,13,1,346,2,10,68,60.0,Сыктывкар-1
9,2011-12-14,2011,12,14,2,347,2,10,69,NaN,Сыктывкар-1


## Mean

### by date independed from points

In [22]:
def calc_mean(src, group_column, target_column):
    mean = src.groupby([group_column])[target_column].mean()
    return src[group_column].apply(lambda x: mean[x])

In [23]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column] = calc_mean(src, column, 'Count')

In [24]:
dst.head(15)

,MeanForMonth,MeanForMonthDay,MeanForThroughDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay
0,112.145788,104.137491,8.0,32.333333,32.333333,90.875335,87.318182,94.327731
1,112.145788,107.172263,88.0,32.333333,32.333333,123.721962,87.318182,91.530201
2,112.145788,107.880699,1.0,32.333333,32.333333,123.640722,87.318182,143.346821
3,104.213605,108.351462,1.0,1.000000,1.000000,87.286115,87.318182,124.594937
4,105.650704,109.919853,NaN,101.277778,18.000000,90.875335,87.318182,78.402299
5,105.650704,107.880699,43.0,101.277778,18.000000,123.721962,87.318182,72.645161
6,105.650704,102.119765,10.0,101.277778,18.000000,142.693482,87.318182,81.650000
7,105.650704,102.062201,1.0,101.277778,18.000000,123.640722,87.318182,103.202532
8,105.650704,100.133333,60.0,101.277778,100.400000,87.286115,87.318182,122.585106
9,105.650704,107.442146,NaN,101.277778,100.400000,87.164051,87.318182,115.263158


### by date depended from points

In [25]:
def map_group_two_key(group, x, y):
    return group[x][y]

def calc_mean_by_columns(src, group_columns, target_column):
    mean = src.groupby(group_columns)[target_column].mean()
    return src[group_columns].apply(lambda x: map_group_two_key(mean, *x), axis=1)

In [26]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column + 'ByPoints'] = calc_mean_by_columns(src, [column, 'Point'], 'Count')

In [27]:
dst.to_csv('data_transform/internal_features.csv')

# All in one

In [28]:
data = pd.read_csv('data_transform/target.csv')
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
internal_features = pd.read_csv('data_transform/internal_features.csv', index_col = 0)

need_columns = date.columns
data[need_columns] = date[need_columns]

need_columns = city.columns
data[need_columns] = city[need_columns]

need_columns = internal_features.columns
data[need_columns] = internal_features[need_columns]

data.to_csv('data_transform/all.csv', index=None)

In [29]:
data

,Count,IsTest,IsTrain,IsValidation,Date,Year,Month,MonthDay,WeekDay,YearDay,...,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,8.0,0,1,0,2011-10-06,2011,10,6,3,278,...,87.318182,94.327731,259.317757,225.255814,8.0,32.333333,32.333333,214.426230,87.318182,171.250000
1,88.0,0,1,0,2011-10-07,2011,10,7,4,279,...,87.318182,91.530201,259.317757,254.404762,88.0,32.333333,32.333333,291.581522,87.318182,185.250000
2,1.0,0,1,0,2011-10-09,2011,10,9,6,281,...,87.318182,143.346821,259.317757,244.022222,1.0,32.333333,32.333333,262.022099,87.318182,223.800000
3,1.0,0,1,0,2011-11-15,2011,11,15,1,318,...,87.318182,124.594937,257.108696,229.959184,1.0,1.000000,1.000000,198.868852,87.318182,190.250000
4,2.0,0,0,1,2011-12-08,2011,12,8,3,341,...,87.318182,78.402299,230.669643,247.133333,NaN,101.277778,18.000000,214.426230,87.318182,227.750000
5,43.0,0,1,0,2011-12-09,2011,12,9,4,342,...,87.318182,72.645161,230.669643,244.022222,43.0,101.277778,18.000000,291.581522,87.318182,129.250000
6,10.0,0,1,0,2011-12-10,2011,12,10,5,343,...,87.318182,81.650000,230.669643,218.857143,10.0,101.277778,18.000000,312.527174,87.318182,116.333333
7,1.0,0,1,0,2011-12-11,2011,12,11,6,344,...,87.318182,103.202532,230.669643,230.153846,1.0,101.277778,18.000000,262.022099,87.318182,163.000000
8,60.0,0,1,0,2011-12-13,2011,12,13,1,346,...,87.318182,122.585106,230.669643,214.871795,60.0,101.277778,100.400000,198.868852,87.318182,197.750000
9,NaN,1,0,0,2011-12-14,2011,12,14,2,347,...,87.318182,115.263158,230.669643,239.975000,NaN,101.277778,100.400000,200.978495,87.318182,257.000000
